In [11]:

import re
import requests

import pdfplumber
import pandas as pd
from collections import namedtuple

#尝试网上的几种提取pdf信息的方法， 先下载PDF
def download_file(url):
    local_filename = url.split('/')[-1]
    
    with requests.get(url) as r:
        with open(local_filename, 'wb') as f:
            f.write(r.content)
        
    return local_filename


ap_url = 'https://www.tabs3.com/support/sample/apreports.pdf'
ap = download_file(ap_url)#download file apreports.pdf

#1.pdfplumber 
with pdfplumber.open(ap) as pdf:
    page = pdf.pages[15]
    pdf_text = page.extract_text()

In [12]:
print(pdf_text)#字句间隔有问题

AccountsPayableReportPack
invoicesenteredandcomparingthetotaltothechecksum,youcanensurethatalltransactions
havebeenenteredandassignedtothepropervendor.
ChangedEntriesand AnyentriesthatareeditedordeletedwillbelistedundertheChangedEntriesorDeletedEntries
DeletedEntries sections.Forchangedentries,theoriginalentrywillbeshownunderthe"Old"heading,andthe
changeswillbereflectedunderthe"New"heading.Whenaninvoiceormanualcheckisdeleted,
theInvoiceAmountisautomaticallychangedtozeroandtheStatusfieldischangedto"D".
TotalTransactionCount Totalnumberoftransactionsonthelistincludingchangesanddeletions.Eachchangedtransaction
(i.e.,botholdandnew)countsasonetransaction.
1Notshownonsamplereport.
-16-


In [13]:
#2.pdfminer
from pdfminer.high_level import extract_pages, extract_text
text=extract_text("apreports.pdf")
print(text)

Accounts Payable Report Pack

Accounts Payable Report Pack

Accounts Payable Report Pack

Copyright © 1987-2023

Software Technology, LLC
1621 Cushman Drive
Lincoln, NE 68512
(402) 423-1440
Tabs3.com

Tabs3, PracticeMaster, Tabs3Pay, and the “pinwheel” symbol (

) are registered trademarks of Software Technology, LLC.

Version 2023 (Tuesday, April 25, 2023)

- 2 -

Accounts Payable Report Pack

List of Reports in Alphabetical Order

1099 Report & 1099 Form

Aged Unpaid Invoice Report

APS Checks

Bank Account List

Cash Requirements Report

Check Register

Criteria Page

Detail Vendor List

Invoice by Vendor List

Invoice by Voucher List

Invoice/Manual Check Entry List

Paid Invoices by Vendor Report

Pre-Check Register

Recurring Entry List

Summary Vendor List

Vendor Analysis Report

Vendor Labels

Verification Lists

Voided Check List

40

23

30

8

25

34

7

11

17

21

13

39

28

9

10

38

13

14

37

- 3 -

Accounts Payable Report Pack

Purposes & Benefits of APS Reports

In [18]:
page=extract_pages("apreports.pdf")
for i in page:
    print(i)
    break

<LTPage(1) 0.000,0.000,612.000,792.000 rotate=0>


In [19]:
#pattern match tool 
pattern=re.compile(r"[a-zA-Z]+,{1}\s{1}")
match=pattern.findall(text)
match

['Technology, ',
 'Lincoln, ',
 'PracticeMaster, ',
 'Pay, ',
 'Technology, ',
 'Tuesday, ',
 'detail,\n',
 'hold, ',
 'numbers,\n',
 'descriptions, ',
 'GLS, ',
 'detail, ',
 'checks, ',
 'order, ',
 'invoices, ',
 'invoices, ',
 'posted, ',
 'invoices, ',
 'date, ',
 'EFTs, ',
 'EFTs, ',
 'MTD, ',
 'invoices,\n',
 'checks, ',
 'added, ',
 'session, ',
 'report, ',
 'week, ',
 'date, ',
 'Jensen, ',
 'Anderson, ',
 'inactive, ',
 'check, ',
 'checks, ',
 'ListJensen, ',
 'Anderson, ',
 'Hold, ',
 'posted, ',
 'Electric, ',
 'ListJensen, ',
 'Anderson, ',
 'zero, ',
 'zero, ',
 'Technology, ',
 'Jensen, ',
 'Anderson, ',
 'Jensen, ',
 'Anderson, ',
 'Parkway, ',
 'Airways, ',
 'Technology, ',
 'prizes, ',
 'awards, ',
 'zero, ',
 'zero, ',
 'zero, ',
 'field, ',
 'example, ',
 'order, ',
 'Labels, ',
 'Labels, ',
 'Labels, ',
 'File, ',
 'File, ',
 'Fee, ',
 'Fee, ',
 'Jensen, ',
 'Anderson, ',
 'Electric, ',
 'Jensen, ',
 'Anderson, ',
 'Jensen, ',
 'Anderson, ',
 'Technology, ',
 'Am

In [25]:
#extract table from pdf 
import tabula 
dfs=tabula.read_pdf("table.pdf",pages='all', multiple_tables=True)
tables={}
for i, df in enumerate(dfs):
        table_name = f"Table {i+1}"
        first_line=df.iloc[0,:]
        # print(first_line.values)
        tables[table_name] = df #这个方法不好：提取的表格会混合周围的文字信息
print(tables.keys()) 
# print(tables['Table 1'])
print(tables['Table 2'])

dict_keys(['Table 1', 'Table 2'])
               评级时间:2023年 6月 19日   优势                      Unnamed: 0
0                            NaN  1.0        煤炭资源及业务规模优势显著。截至 2022年底,
1                本次评级使用的评级方法、模型:  NaN  公司煤炭资源总量 908.72亿吨,可采储量为 180.55
2                          名称 版本  NaN       亿吨,煤炭产能 3.24亿吨/年。同时,公司具备年
3         煤炭企业信用评级方法 V4.0.202208  NaN                             NaN
4                            NaN  NaN     产甲醇 531万吨、醋酸 100万吨、尿素 52万吨、
5  煤炭企业主体信用评级模型(打分表) V4.0.202208  NaN                             NaN
6                            NaN  NaN                  焦炭 563万吨的生产能力。
7     注:上述评级方法和评级模型均已在联合资信官网公开披露  NaN                             NaN
8                            NaN  2.0           整体经营业绩水平大幅提升,经营活动现金净流


In [27]:
import fitz #PyMUPDF
import PIL.Image #pillow 
import io

#tool for extracting image, saved as jpeg 

pdf=fitz.open("Online_ticket.pdf")#extract images from pdf
counter =1
for i in range(len(pdf)):
    page = pdf[i]
    images=page.get_images()
    for image in images:
        base_img = pdf.extract_image(image[0])
        image_data = base_img["image"]
        img = PIL.Image.open(io.BytesIO(image_data))
        extension=base_img["ext"]
        img.save(open(f"image{counter}.{extension}", "wb"))
        counter+=1